In [0]:
# training code for DuoRC dataset

In [2]:
# TODO(lanzhzh): Add support for 2.x.
%tensorflow_version 1.x
import os
import pprint
import json
import tensorflow as tf

assert "COLAB_TPU_ADDR" in os.environ, "ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!"
TPU_ADDRESS = "grpc://" + os.environ["COLAB_TPU_ADDR"] 
TPU_TOPOLOGY = "2x2"
print("TPU address is", TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.

TensorFlow 1.x selected.
TPU address is grpc://10.15.87.250:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8306339390939790152),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17065015445630684349),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8395844443371443076),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11735596849016995926),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 11369935624475566570),
 _DeviceAttributes(/job:tp

In [3]:
#TODO(lanzhzh): Add pip support
import sys

!test -d albert || git clone https://github.com/google-research/albert albert
if not 'albert' in sys.path:
  sys.path += ['albert']
  
!pip install sentencepiece

Cloning into 'albert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 226.59 KiB | 3.02 MiB/s, done.
Resolving deltas: 100% (225/225), done.
     |████████████████████████████████| 1.0MB 3.0MB/s 


In [5]:
BUCKET = "albert-train-squad" 
TASK = "DuoRC_self"
# Available pretrained model checkpoints:
#   base, large, xlarge, xxlarge
ALBERT_MODEL = 'base' 

***** Model output directory: gs://albert-train-squad/albert-tfhub/squad_models/DuoRC_self *****


In [6]:
# Download squad data.
 TASK_DATA_DIR = 'DuoRC_data'
 !mkdir $TASK_DATA_DIR \
 && cd $TASK_DATA_DIR \

# ! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
# !python download_glue_repo/download_glue_data.py --data_dir=$TASK_DATA_DIR --tasks=$TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))

# ALBERT_MODEL_HUB = 'https://tfhub.dev/google/albert_' + ALBERT_MODEL + '/3'

***** Task data directory: DuoRC_data *****


In [7]:
!python albert/modeling.py



!pip install -r albert/requirements.txt

     |████████████████████████████████| 92kB 2.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4913f53180da2972891970c6de7d299b34c99da26392a693210a744f65aa0b3f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [19]:
# os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs"\
  --train_file="DuoRC_data/DuoRC_Self_train.json"\
  --predict_file="DuoRC_data/DuoRC_Self_dev.json"\
  --vocab_file="albert_base/30k-clean.vocab" \
  --albert_hub_module_handle=https://tfhub.dev/google/albert_base/1\
  --spm_model_file="albert_base/30k-clean.model"\
  --do_lower_case \
  --max_seq_length=384\
  --doc_stride=128\
  --max_query_length=64 \
  --do_predict \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30\
  --train_feature_file="DuoRC_train_feature_file.tf" \
  --predict_feature_file="DuoRC_predict_feature_file.tf" \
  --predict_feature_left_file="DuoRC_predict_left_feature_file.tf" \
  --do_train \
  

I0412 12:45:15.516308 139658215733120 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I0412 12:45:17.326176 139658215733120 saver.py:1503] Saver not created because there are no variables in the graph to restore
2020-04-12 12:45:17.412546: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-12 12:45:17.414552: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-12 12:45:17.414599: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2fb4273874f7): /proc/driver/nvidia/version does not exist
2020-04-12 12:45:17.421376: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-12 12:45:17.421650: I tensorflow/compiler/xla/service/servi

In [20]:
!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs" \
  --train_file="DuoRC_data/DuoRC_Self_train.json" \
  --predict_file="DuoRC_data/DuoRC_Self_dev.jsonn" \
  --vocab_file="albert_base/30k-clean.vocab" \
  --train_feature_file="DuoRC_train_feature_file.tf" \
  --predict_feature_file="DuoRC_predict_feature_file.tf" \
  --predict_feature_left_file="DuoRC_predict_left_feature_file.tf" \
  --spm_model_file="albert_base/30k-clean.model" \
  --albert_hub_module_handle=https://tfhub.dev/google/albert_base/1\
  --do_train\
  --do_lower_case \
  --max_seq_length=384 \
  --doc_stride=128 \
  --max_query_length=64 \
  --do_predict \
  --use_gpu \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30 
    # https://tfhub.dev/google/albert_base/1
    #  --init_checkpoint="albert_base/model.ckpt-best.index" \
    #   --do_train \

I0412 12:46:41.504272 139743838541696 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I0412 12:46:43.376690 139743838541696 saver.py:1503] Saver not created because there are no variables in the graph to restore
2020-04-12 12:46:43.481049: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-12 12:46:43.525836: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-12 12:46:43.525924: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2fb4273874f7): /proc/driver/nvidia/version does not exist
2020-04-12 12:46:43.565202: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-12 12:46:43.565787: I tensorflow/compiler/xla/service/servi

In [0]:
!python -m albert.run_squad_v2 \
  --albert_config_file="albert_base/albert_config.json" \
  --output_dir="albert/outputs"\
  --train_file="DuoRC_data/DuoRC_Self_train.json"\
  --predict_file="DuoRC_data/DuoRC_Self_dev.json"\
  --vocab_file="albert_base/30k-clean.vocab" \
  --albert_hub_module_handle=https://tfhub.dev/google/albert_base/1\
  --spm_model_file="albert_base/30k-clean.model"\
  --do_lower_case \
  --max_seq_length=384\
  --doc_stride=128\
  --max_query_length=64 \
  --do_predict \
  --train_batch_size=48 \
  --predict_batch_size=8 \
  --learning_rate=5e-5 \
  --num_train_epochs=5.0 \
  --warmup_proportion=.1 \
  --save_checkpoints_steps=5000 \
  --n_best_size=20 \
  --max_answer_length=30\
  --train_feature_file="DuoRC_train_feature_file.tf" \
  --predict_feature_file="DuoRC_predict_feature_file.tf" \
  --predict_feature_left_file="DuoRC_predict_left_feature_file.tf" \

I0412 12:47:59.573875 139862517847936 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I0412 12:48:01.436006 139862517847936 saver.py:1503] Saver not created because there are no variables in the graph to restore
2020-04-12 12:48:01.542689: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-04-12 12:48:01.588132: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-04-12 12:48:01.588232: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2fb4273874f7): /proc/driver/nvidia/version does not exist
2020-04-12 12:48:01.626567: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2020-04-12 12:48:01.627000: I tensorflow/compiler/xla/service/servi